In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key

tavily_api_key = os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [41]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

conn = sqlite3.connect("checkpoints.sqlite", check_same_thread=False)
memory = SqliteSaver(conn)

In [18]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [42]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [20]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [21]:
thread = {"configurable": {"thread_id": "1"}}

In [43]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Bsy371M3xAr1cFYedr1OufFd', 'function': {'arguments': '{"query":"current weather San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIAktTPfrry8ZgXd5SEQW6J1Xr2zN', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3b7176fe-c346-4db8-a494-0d42c2e84362-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather San Francisco'}, 'id': 'call_Bsy371M3xAr1cFYedr1OufFd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 22, 

In [44]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZBXzId0J2DK9PHZUejex3ZlJ', 'function': {'arguments': '{"query":"current weather Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 1419, 'total_tokens': 1441, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1280}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIAoVKtzsuwpzsZ24lDN76DFxnIbM', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8acde81e-8080-4c33-b85c-4b91eb788950-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather Los Angeles'}, 'id': 'call_ZBXzId0J2DK9PHZUejex3ZlJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1419, 'outpu

In [45]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Los Angeles is currently warmer than San Francisco. Los Angeles has a temperature of 12.2°C (54.0°F) compared to San Francisco's 7.8°C (46.0°F).", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 2057, 'total_tokens': 2101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1920}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIArvdSSVSbreD26ERnSqmkvk5aei', 'finish_reason': 'stop', 'logprobs': None}, id='run-888d98b2-2fdb-4505-9c09-4712ace7d993-0', usage_metadata={'input_tokens': 2057, 'output_tokens': 44, 'total_tokens': 2101, 'input_token_details': {'audio': 0, 'cache_read': 1920}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [46]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please provide more context or specify the locations, items, or time periods you are comparing to determine which one is warmer?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 149, 'total_tokens': 177, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIAsgJKUETagVWeEkh5Qu537FzYzF', 'finish_reason': 'stop', 'logprobs': None}, id='run-b294e563-7b0c-46a1-b1a5-049f4efa6e81-0', usage_metadata={'input_tokens': 149, 'output_tokens': 28, 'total_tokens': 177, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [57]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
import aiosqlite

# conn = sqlite3.connect("checkpoints.sqlite", check_same_thread=False)
# memory = SqliteSaver(conn)

conn = aiosqlite.connect("checkpoints.sqlite", check_same_thread = False)
memory = AsyncSqliteSaver(conn)
# async with aiosqlite.connect("checkpoints.db") as conn:
#     memory = AsyncSqliteSaver(conn)

abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [59]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="")

The current weather in San Francisco is partly cloudy with a temperature of approximately 9.4°C (48.9°F). It is nighttime, with wind blowing at 10.7 mph from the northwest. The humidity level is around 83%, and the conditions have a wind chill factor making it feel like 6.9°C (44.4°F). Visibility is approximately 16 kilometers.